In [1]:
from pyalgotrade.tools import yahoofinance;
yahoofinance.download_daily_bars('orcl', 2000, 'orcl-2000.csv')


In [2]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info(bar.getClose())

In [3]:
# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("orcl", "orcl-2000.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

2000-01-03 00:00:00 strategy [INFO] 118.125
2000-01-04 00:00:00 strategy [INFO] 107.6875
2000-01-05 00:00:00 strategy [INFO] 102.0
2000-01-06 00:00:00 strategy [INFO] 96.0
2000-01-07 00:00:00 strategy [INFO] 103.375
2000-01-10 00:00:00 strategy [INFO] 115.75
2000-01-11 00:00:00 strategy [INFO] 112.375
2000-01-12 00:00:00 strategy [INFO] 105.625
2000-01-13 00:00:00 strategy [INFO] 105.0625
2000-01-14 00:00:00 strategy [INFO] 106.8125
2000-01-18 00:00:00 strategy [INFO] 111.25
2000-01-19 00:00:00 strategy [INFO] 57.125
2000-01-20 00:00:00 strategy [INFO] 59.25
2000-01-21 00:00:00 strategy [INFO] 59.6875
2000-01-24 00:00:00 strategy [INFO] 54.1875
2000-01-25 00:00:00 strategy [INFO] 56.4375
2000-01-26 00:00:00 strategy [INFO] 55.0625
2000-01-27 00:00:00 strategy [INFO] 51.8125
2000-01-28 00:00:00 strategy [INFO] 47.375
2000-01-31 00:00:00 strategy [INFO] 49.953098
2000-02-01 00:00:00 strategy [INFO] 54.0
2000-02-02 00:00:00 strategy [INFO] 54.3125
2000-02-03 00:00:00 strategy [INFO] 56.68

In [2]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        strategy.BacktestingStrategy.__init__(self, feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    # Load the yahoo feed from the CSV file
    feed = yahoofeed.Feed()
    feed.addBarsFromCSV("orcl", "orcl-2000.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print "Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity()

run_strategy(20)

2000-02-02 00:00:00 strategy [INFO] BUY at $25.22
2000-03-31 00:00:00 strategy [INFO] SELL at $36.82
2000-04-07 00:00:00 strategy [INFO] BUY at $38.42
2000-04-12 00:00:00 strategy [INFO] SELL at $35.78
2000-04-28 00:00:00 strategy [INFO] BUY at $36.04
2000-05-04 00:00:00 strategy [INFO] SELL at $34.58
2000-05-08 00:00:00 strategy [INFO] BUY at $34.58
2000-05-09 00:00:00 strategy [INFO] SELL at $33.83
2000-05-15 00:00:00 strategy [INFO] BUY at $34.18
2000-05-19 00:00:00 strategy [INFO] SELL at $33.06
2000-05-31 00:00:00 strategy [INFO] BUY at $33.63
2000-06-26 00:00:00 strategy [INFO] SELL at $36.85
2000-06-27 00:00:00 strategy [INFO] BUY at $37.82
2000-06-30 00:00:00 strategy [INFO] SELL at $36.90
2000-07-03 00:00:00 strategy [INFO] BUY at $37.25
2000-07-05 00:00:00 strategy [INFO] SELL at $35.27
2000-07-21 00:00:00 strategy [INFO] BUY at $35.55
2000-07-24 00:00:00 strategy [INFO] SELL at $35.41
2000-07-27 00:00:00 strategy [INFO] BUY at $34.87
2000-07-28 00:00:00 strategy [INFO] SELL 

Final portfolio value: $1075.76
